In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
import matplotlib.pyplot as plt
drive.mount('/content/drive')

In [ ]:
# This code was run individually for each dataset (cryptocurrency) by replacing all ETH with BTC, LTC and XRP.

# Starting point of a dataset
start = 1551225600000
# How many minutes is going to be in each time step
discretization = 5
i = 1
new_array = []
for k in range(0, 13):
  data = pd.read_csv('/content/drive/My Drive/ETH_DL/'+str(k)+'.csv')
  data['ask/bid'] = 0
  # m - is answer to question "Was the buyer the maker", hence if he was, it is a bid, else it is an ask
  data['ask/bid'][data['m'] == True] = 'bid'
  data['ask/bid'][data['m'] == False] = 'ask'
  max_time = max(data['T'])
  while start+(i-1)*60000*discretization<max_time:
    print(i)
    if i == 1:
      # Choosing all the prices in an interval
      minute_piece = data[(data['T']>=start+(i-1)*60000*discretization) & (data['T']<start+i*60000*discretization)]
      # Finding best asks as the orders of sellers with the smallest price
      ask = minute_piece[minute_piece['ask/bid'] == 'ask'].nsmallest(10, 'p')
      # Finding the best bids as the orders of buyers with the highest price
      bid = minute_piece[minute_piece['ask/bid'] == 'bid'].nlargest(10, 'p')
      # Concatenating them
      new_array = np.expand_dims(np.concatenate((ask[['p', 'q']].to_numpy(), bid[['p', 'q']].to_numpy()), axis = 1), 0)
    else:
      # Choosing all the prices in an interval
      minute_piece = data[(data['T']>=start+(i-1)*60000*discretization) & (data['T']<start+i*60000*discretization)]
      # Finding best asks as the orders of sellers with the smallest price
      ask = minute_piece[minute_piece['ask/bid'] == 'ask'].nsmallest(10, 'p')
      # Finding the best bids as the orders of buyers with the highest price
      bid = minute_piece[minute_piece['ask/bid'] == 'bid'].nlargest(10, 'p')
      if ask.shape == (10, 10) and bid.shape == (10, 10):
        # Concatenating them in case they are complete (have the standard shape)
        point = np.expand_dims(np.concatenate((ask[['p', 'q']].to_numpy(), bid[['p', 'q']].to_numpy()), axis = 1), 0)
        new_array = np.concatenate((new_array, point), axis = 0)
      else:
        # If they are incomplete, the arrays cannot be concatenated (kind of missed value), hence we have to replace incomplete arrays with the previous ones.
        new_array = np.concatenate((new_array, np.expand_dims(new_array[-1], 0)), axis = 0)
    i += 1
# Transponising each datapoint for conviniency
unnormalised = []
for i in new_array:
  unnormalised += [i.T]
unnormalised = np.array(unnormalised)
# Reversing the bids to get the appropriate form for further input into the model
unnormalised_sorted = []
for i in unnormalised:
  unnormalised_sorted += [[i[0], i[1], np.flip(i[2]), np.flip(i[3])]]
unnormalised_sorted = np.array(unnormalised_sorted)
unnormalised = unnormalised_sorted
np.save('unnormalised_ETHUSDT', unnormalised)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


Streaming output truncated to the last 5000 lines.
100122
100123
100124
100125
100126
100127
100128
100129
100130
100131
100132
100133
100134
100135
100136
100137
100138
100139
100140
100141
100142
100143
100144
100145
100146
100147
100148
100149
100150
100151
100152
100153
100154
100155
100156
100157
100158
100159
100160
100161
100162
100163
100164
100165
100166
100167
100168
100169
100170
100171
100172
100173
100174
100175
100176
100177
100178
100179
100180
100181
100182
100183
100184
100185
100186
100187
100188
100189
100190
100191
100192
100193
100194
100195
100196
100197
100198
100199
100200
100201
100202
100203
100204
100205
100206
100207
100208
100209
100210
100211
100212
100213
100214
100215
100216
100217
100218
100219
100220
100221
100222
100223
100224
100225
100226
100227
100228
100229
100230
100231
100232
100233
100234
100235
100236
100237
100238
100239
100240
100241
100242
100243
100244
100245
100246
100247
100248
100249
100250
100251
100252
100253
100254
100255
100256
1002

In [ ]:
# 5 days z-score normalization
normalised = []
for hour_global in range(5*24*12+1, unnormalised.shape[0]):
  features = []
  for i in range(4):
    temp = []
    for hour_local in range(5*24*12):
      temp += [unnormalised[hour_global - hour_local][i]]
    temp = np.array(temp).reshape(-1)
    mean = temp.mean()
    std = temp.std()
    features += [(unnormalised[hour_global][i] - mean) / std]
  normalised += [features]
normalised = np.array(normalised)
np.save('normalised_ETHUSDT', normalised)

In [ ]:
k = 1
alpha = 0.001

# Mid prices
mid_prices = []
for point in unnormalised:
  mid_prices += [(point[0][0]+point[2][0])/2]

# m+ and m- mean prices
m_minus = []
for i in range(5*24*12+1, unnormalised.shape[0]-k):
  m_minus += [sum(mid_prices[i - k : i]) / k]
m_minus = np.array(m_minus)  
m_plus = []
for i in range(5*24*12+1, unnormalised.shape[0]-k):
  m_plus += [sum(mid_prices[i : i + k]) / k]
m_plus = np.array(m_plus)  
# Relative changes
l_t = (m_plus - m_minus) / m_minus

# Labels
label = []
for i in l_t:
  if i >= alpha:
    label += [0]
  elif i <= -alpha:
    label += [2]
  else:
    label += [1]
label = np.array(label)

In [ ]:
np.save('input_ETHUSDT_'+str(k), normalised.reshape(-1, 40)[:-k])
np.save('labels_ETHUSDT_'+str(k), label)
!cp input_ETHUSDT_1.npy "drive/My Drive/Processed_datasets"
!cp labels_ETHUSDT_1.npy "drive/My Drive/Processed_datasets"